In [16]:
# Cellule 1 : Import + paramètres globaux

import os
import pandas as pd
import folium
from IPython.display import display

base_path = "donnee_accident/Donnee_"
annees = [2005, 2008, 2011, 2014, 2017, 2020, 2023]

# Bordeaux : dep = 330, com = 33063 (code commune)
dep_bordeaux = "330"
com_bordeaux = "33063"

# Couleurs par année pour la carte
colors = {
    2005: "blue",
    2008: "green",
    2011: "purple",
    2014: "orange",
    2017: "darkred",
    2020: "cadetblue",
    2023: "darkgreen"
}

In [17]:
# Cellule 2 : Chargement et fusion des données Bordeaux

dfs_bordeaux = []

for an in annees:
    chemin_carac = f"{base_path}{an}/caracteristiques-{an}.csv"
    chemin_lieux = f"{base_path}{an}/lieux-{an}.csv"
    
    if not (os.path.exists(chemin_carac) and os.path.exists(chemin_lieux)):
        print(f"⛔ Fichiers manquants pour {an}")
        continue

    try:
        carac = pd.read_csv(chemin_carac, sep=";", encoding="latin1", on_bad_lines="skip", low_memory=False)
        lieux = pd.read_csv(chemin_lieux, sep=";", encoding="utf-8", on_bad_lines="skip")

        # Nettoyage noms colonnes
        carac.columns = carac.columns.str.strip().str.replace('"', '')
        lieux.columns = lieux.columns.str.strip().str.replace('"', '')

        # Debug
        print(f"\n📂 Année {an} | Colonnes carac : {list(carac.columns)}")
        print(f"📂 Année {an} | Colonnes lieux : {list(lieux.columns)}")

        # Fusion
        df = pd.merge(carac, lieux, on="Num_Acc", how="left")

        # Normalisation codes
        df["dep"] = df["dep"].astype(str).str.zfill(3)
        df["com"] = df["com"].astype(str).str.zfill(5)

        # Filtrer Bordeaux
        filtre = (df["dep"] == dep_bordeaux) | (df["com"] == com_bordeaux)
        df_bdx = df[filtre].copy()
        df_bdx["annee"] = an

        dfs_bordeaux.append(df_bdx)
        print(f"✔️ {an} : {len(df_bdx)} accidents à Bordeaux")

    except Exception as e:
        print(f"❌ Erreur pour {an} : {e}")

if dfs_bordeaux:
    df_bordeaux = pd.concat(dfs_bordeaux, ignore_index=True)
    print(f"\nTotal accidents Bordeaux (toutes années) : {len(df_bordeaux)}")
else:
    print("\nAucune donnée Bordeaux chargée")



📂 Année 2005 | Colonnes carac : ['Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep']
📂 Année 2005 | Colonnes lieux : ['Num_Acc,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1']
❌ Erreur pour 2005 : 'Num_Acc'

📂 Année 2008 | Colonnes carac : ['Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep']
📂 Année 2008 | Colonnes lieux : ['Num_Acc,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1']
❌ Erreur pour 2008 : 'Num_Acc'

📂 Année 2011 | Colonnes carac : ['Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep']
📂 Année 2011 | Colonnes lieux : ['Num_Acc,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1']
❌ Erreur pour 2011 : 'Num_Acc'

📂 Année 2014 | Colonnes carac : ['Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep']
📂 Année 2014 | Colonnes lieux : ['Num_Acc,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartp

C:\Users\lukas\AppData\Local\Temp\ipykernel_12804\4054816478.py:15: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  lieux = pd.read_csv(chemin_lieux, sep=";", encoding="utf-8", on_bad_lines="skip")


In [18]:
# Cellule 3 : Nettoyage des coordonnées

def convert_coord(val):
    """Convertit coordonnées GPS du format entier (ex 4415750) en float (ex 44.15750)"""
    try:
        val_str = str(val).replace(",", ".").strip()
        if "." in val_str:
            return float(val_str)
        else:
            # Cas coordonnées sans virgule, conversion manuelle
            val_str = val_str.zfill(7)  # ajouter des 0 si besoin
            if len(val_str) == 7:
                return float(val_str[:2] + "." + val_str[2:])
            elif len(val_str) == 6:
                return float(val_str[:1] + "." + val_str[1:])
            else:
                return float(val_str)
    except:
        return None

df_bordeaux["lat"] = df_bordeaux["lat"].apply(convert_coord)
df_bordeaux["long"] = df_bordeaux["long"].apply(convert_coord)

# Supprimer points sans coordonnée valide
df_bordeaux = df_bordeaux.dropna(subset=["lat", "long"])

print(f"Après nettoyage, {len(df_bordeaux)} accidents avec coordonnées valides.")


Après nettoyage, 543 accidents avec coordonnées valides.


In [2]:
# Cellule 4 : Création carte interactive Folium

map_bdx = folium.Map(location=[44.8378, -0.5792], zoom_start=12)

for an in sorted(df_bordeaux["annee"].unique()):
    subset = df_bordeaux[df_bordeaux["annee"] == an]
    color = colors.get(an, "gray")
    for _, row in subset.iterrows():
        popup_txt = f"Année : {row['annee']}<br>Adresse : {row.get('adr', 'N/A')}<br>Heure : {str(row.get('hrmn', 'N/A')).zfill(4)}"
        folium.CircleMarker(
            location=[row["lat"], row["long"]],
            radius=3,
            color=color,
            fill=True,
            fill_opacity=0.6,
            popup=folium.Popup(popup_txt, max_width=300)
        ).add_to(map_bdx)

# Sauvegarde
map_bdx.save("carte_accidents_bordeaux.html")

# Affichage dans Jupyter
display(map_bdx)

df.shape

NameError: name 'folium' is not defined